##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.X - Multi-tool with the Multimodal Live API

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/LiveAPI_plotting_and_mapping.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this notebook you will learn how to use tools, including charting tools, Google Search and code execution in the [Gemini 2](https://ai.google.dev/gemini-api/docs/models/gemini-v2) Multimodal Live API. For an overview of new capabilities refer to the [Gemini 2 docs](https://ai.google.dev/gemini-api/docs/models/gemini-v2).

This notebook is written in Python and uses the secure Websockets protocol directly, it does *not* use the GenAI SDK.

If you aren't looking for code, and just want to try multimedia streaming use [Live API in Google AI Studio](https://aistudio.google.com/app/live).

## Get set up

In [ ]:
%pip install -q 'websockets~=14.0' altair

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 5.1 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](../quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
import os
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

Multimodal Live API are a new capability introduced with the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) model. It won't work with previous generation models.

You also need to set the client version to `v1alpha`.

In [ ]:
uri = f"wss://generativelanguage.googleapis.com/ws/google.ai.generativelanguage.v1beta.GenerativeService.BidiGenerateContent?key={GOOGLE_API_KEY}"
model = "models/gemini-2.5-flash-native-audio-preview-09-2025"

### Set up some helpers

Before interacting with the API, define some helpers that you'll need in this codelab.

In this notebook, you'll be buffering the streamed PCM audio responses, so create a context manager to wrap the PCM audio data in a wave audio file with the relevant audio parameters. This way, you can play the audio back directly within Colab.

In [ ]:
import contextlib
import wave

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
  """Define a wave context manager using the audio parameters supplied."""
  with wave.open(filename, "wb") as wf:
    wf.setnchannels(channels)
    wf.setsampwidth(sample_width)
    wf.setframerate(rate)
    yield wf

Use a custom logger so you can easily toggle the log level in order to see in-flight requests and responses from the API.

In [ ]:
import logging
logger = logging.getLogger("Live")
# Switch to "DEBUG" to see the in-flight requests & responses
logger.setLevel("INFO")

### Define connection functions

This code defines some functions that will connect to (`quick_connect`), execute and handle prompts (`run`) and handle specific server responses (`handle_tool_call`, `handle_server_content`).

This code uses the [websockets](https://pypi.org/project/websockets) PyPI package, specifically the async interface available in 14.0 and will not work with significantly older packages.

In [ ]:
import asyncio
import base64
import json
import time

from websockets.asyncio.client import connect
from IPython import display


async def setup(ws, modality, tools):
  """Perform a setup handshake to configure the conversation."""
  setup = {
      "setup": {
          "model": model,
          "tools": tools,
          "generation_config": {
              "response_modalities": [modality]
          }
      }
  }
  setup_json = json.dumps(setup)
  logger.debug(">>> " + setup_json)
  await ws.send(setup_json)

  setup_response = json.loads(await ws.recv())
  logger.debug("<<< " + json.dumps(setup_response))

async def send(ws, prompt):
  """Send a user content message (only text is supported)."""
  msg = {
    "client_content": {
      "turns": [{"role": "user", "parts": [{"text": prompt}]}],
      "turn_complete": True,
    }
  }
  json_msg = json.dumps(msg)
  logger.debug(">>> " + json_msg)
  await ws.send(json_msg)


def handle_server_content(wf, server_content):
  """Handle any server content messages, e.g. incoming audio or text."""
  audio = False
  model_turn = server_content.pop("modelTurn", None)
  if model_turn:
    text = model_turn["parts"][0].pop("text", None)
    if text:
      print(text, end='')

    inline_data = model_turn['parts'][0].pop('inlineData', None)
    if inline_data:
      print('.', end='')
      b64data = inline_data['data']
      pcm_data = base64.b64decode(b64data)
      wf.writeframes(pcm_data)
      audio = True

  turn_complete = server_content.pop('turnComplete', None)
  return turn_complete, audio


async def handle_tool_call(ws, tool_call, responses):
  """Process an incoming tool call request, returning a response."""
  logger.debug("<<< " + json.dumps(tool_call))
  for fc in tool_call['functionCalls']:

    if fc['name'] in responses:
      # Use a response from `responses` if provided.
      result_entry = responses[fc['name']]
      # If it's a function, actuall call it.
      if callable(result_entry):
        result = result_entry(**fc['args'])
    else:
      # Otherwise it's a stub, just say "OK"
      result = {'string_value': 'ok'}

    msg = {
      'tool_response': {
          'function_responses': [{
              'id': fc['id'],
              'name': fc['name'],
              'response': {'result': result}
          }]
        }
    }
    json_msg = json.dumps(msg)
    logger.debug(">>> " + json_msg)
    await ws.send(json_msg)


@contextlib.asynccontextmanager
async def quick_connect(modality='TEXT', tools=()):
  """Establish a connection and keep it open while the context is active."""
  async with connect(uri, additional_headers={"Content-Type": "application/json"}) as ws:
    await setup(ws, modality, tools)
    yield ws


audio_lock = time.time()

async def run(ws, prompt, responses=()):
  """Send the provided prompt and handle the streamed response."""
  print('>', prompt)
  await send(ws, prompt)

  audio = False
  filename = 'audio.wav'
  with wave_file(filename) as wf:
    async for raw_response in ws:
      response = json.loads(raw_response.decode())
      logger.debug("<<< " + str(response)[:150])

      server_content = response.pop("serverContent", None)
      if server_content:
        turn_complete, a = handle_server_content(wf, server_content)
        audio = audio or a

        if turn_complete:
          print()
          print('<Turn complete>')
          break

      tool_call = response.pop('toolCall', None)
      if tool_call:
        await handle_tool_call(ws, tool_call, responses)

  if audio:
    global audio_lock
    # Sleep before playing audio to make sure it doesn't play over an existing clip.
    if (delta := audio_lock - time.time()) > 0:
      print('Pausing for audio to complete...')
      await asyncio.sleep(delta + 1.0)  # include a buffer so there's a breather

    display.display(display.Audio(filename, autoplay=True))
    audio_lock = time.time() + (wf.getnframes() / wf.getframerate())

## Use the API

### One-turn example

Now, let's see how all the pieces you've defined fit together in a simple example. You'll send a single prompt to the API and observe the response.

This example uses the `quick_connect` context manager to create a connection to the API. As long as you're inside the async with block, the connection remains active and is accessible through the ws variable. Then use the run function to send our prompt and process the API's response.


Make a simple request to understand how the above code works. A connection is created through a context manager using `quick_connect`, and while the context is active, the web-socket connection is stored in `ws`, and passed to subsequent `run` calls that execute the prompts.

Note that you can change the modality from `AUDIO` to `TEXT`. and adjust the prompt.

In [ ]:
tools = [
    {'google_search': {}},
    {'code_execution': {}},
]

async def go():
  async with quick_connect(tools=tools, modality="TEXT") as ws:
    await run(ws, "Please find the last 5 Denis Villeneuve movies and look up their runtimes and the year published.")

logger.setLevel('INFO')
await go()

> Please find the last 5 Denis Villeneuve movies and look up their runtimes and the year published.
Based on the search results, the last 5 Denis Villeneuve movies are:

1.  *Dune: Part Two* (2024)
2.  *Dune: Part One* (2021)
3.  *Blade Runner 2049* (2017)
4.  *Arrival* (2016)
5.  *Sicario* (2015)

Now, let's find the runtimes for these movies.
Here's a summary of the last 5 Denis Villeneuve movies, their release year, and runtime:

*   **Dune: Part Two** (2024): 166 minutes (2 hours 46 minutes)
*   **Dune: Part One** (2021): 155 minutes (2 hours 35 minutes)
*   **Blade Runner 2049** (2017): 163 minutes (2 hours 43 minutes)
*   **Arrival** (2016): 116 minutes (1 hour 56 minutes)
*   **Sicario** (2015): 121 minutes (2 hours 1 minute)
<Turn complete>


### Complex multi-tool example

Now define additional tools. Add a tool for charting by defining a schema (in `altair_fns`), a function to execute (`render_altair`) and connect the two using the `tool_calls` mapping.

The charting tool used here is [Vega-Altair](https://pypi.org/project/altair/), a "declarative statistical visualization library for Python". Altair supports chart persistance using JSON, which you will expose as a tool so that the Gemini model can produce a chart.

The helper code defined earlier will run as soon as it can, but audio takes some time to play so you may see output from later turns displayed before the audio has played.

In [ ]:
import altair as alt
from google.api_core import retry


def apply_altair_theme(altair_json: str, theme: str) -> str:
  chart = alt.Chart.from_json(altair_json)
  with alt.themes.enable(theme):
    themed_altair_json = chart.to_json()
  return themed_altair_json


@retry.Retry()
def render_altair(altair_json: str, theme: str = "default"):
  themed_altair_json = apply_altair_theme(altair_json, theme)
  chart = alt.Chart.from_json(themed_altair_json)
  chart.display()

  return {'string_value': 'ok'}


altair_fns = [
  {
    'name': 'render_altair',
    'description': 'Displays an Altair chart in JSON format.',
    'parameters': {
      'type': 'OBJECT',
      'properties': {
        'altair_json': {
            'type': 'STRING',
            'description': 'JSON STRING representation of the Altair chart to render. Must be a string, not a json object',
        },
        'theme': {
            'type': 'STRING',
            'description': 'Altair theme. Choose from one of "dark", "ggplot2", "default", "opaque".',
        },
      },
    },
  },
]

tool_calls = {
    'render_altair': render_altair,
}

Now put that all together into a chat conversation. This code opens a streaming session (with `quick_connect`), and each `run` invocation will send the text prompt, read the streamed response (and buffer if it's audio), handle any server responses (such as tool calls) and finally return once the end-of-turn signal has been sent.

By sequencing multiple `run` calls within a `quick_connect` session, you are executing a multi-turn, streamed, conversation. Once the code reaches the end of the `quick_connect` block, the session is terminated.

In [ ]:
tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': altair_fns},
]

async def go():
  async with quick_connect(tools=tools, modality="AUDIO") as ws:

    # Google Search
    await run(ws, "Please find the last 5 Denis Villeneuve movies and find their runtimes.")
    # Code execution
    await run(ws, "Can you write some code to work out which has the longest and shortest runtimes?")
    # Tool use
    await run(ws, "Now can you plot them in a line chart showing the year on the x-axis and runtime on the y-axis?", responses=tool_calls)
    # Tool use - this step takes user input, so you can ask the model to tweak the chart to your liking.
    # Try changing to dark mode, or lay out the data differently.
    await run(ws, input('Any requests? > '), responses=tool_calls)


logger.setLevel('INFO')
await go()

> Please find the last 5 Denis Villeneuve movies and find their runtimes.
.......................................................................................................................
<Turn complete>


> Can you write some code to work out which has the longest and shortest runtimes?
.....................................................
<Turn complete>
Pausing for audio to complete...


> Now can you plot them in a line chart showing the year on the x-axis and runtime on the y-axis?


<ipython-input-10-14ecbacb69f1>:7: AltairDeprecationWarning: 
Deprecated since `altair=5.5.0`. Use altair.theme instead.
Most cases require only the following change:

    # Deprecated
    alt.themes.enable('quartz')

    # Updated
    alt.theme.enable('quartz')

If your code registers a theme, make the following change:

    # Deprecated
    def custom_theme():
        return {'height': 400, 'width': 700}
    alt.themes.register('theme_name', custom_theme)
    alt.themes.enable('theme_name')

    # Updated
    @alt.theme.register('theme_name', enable=True)
    def custom_theme():
        return alt.theme.ThemeConfig(
            {'height': 400, 'width': 700}
        )

See the updated User Guide for further details:
    https://altair-viz.github.io/user_guide/api.html#theme
    https://altair-viz.github.io/user_guide/customization.html#chart-themes
  with alt.themes.enable(theme):


alt.Chart(...)

....................................................
<Turn complete>


Any requests? > can you add the movie names into the dots as also make it in a dark theme?
> can you add the movie names into the dots as also make it in a dark theme?


alt.LayerChart(...)

.............................
<Turn complete>


### Maps example

For this example you will use the [Google Maps Static API](https://developers.google.com/maps/documentation/maps-static) to draw on a map during the conversation. You'll need to [make sure your API key is enabled for the Google Maps Static API](https://developers.google.com/maps/documentation/maps-static/get-api-key). It can be the same API key as you used for the Gemini API, or a new one, as long as the Static Maps API is enabled.

Add the key in Colab Secrets, or add it in the code directly (`MAPS_API_KEY = 'AIza...'`).

In [ ]:
from google.colab import userdata
MAPS_API_KEY = userdata.get('MAPS_API_KEY')

The following cell is hidden by default, but needs te be run. It comtains the function schema for the `draw_map` function, including some documentation on how to draw markers with the Google Maps API.

Note that the model needs to produce a fairly complex set of parameters in order to call `draw_map`, including defining a center-point for the map, an integer zoom level and custom marker styles and locations.

In [ ]:
# @title Map tool schema (run this cell)

map_fns = [
  {
    'name': 'draw_map',
    'description': 'Render a Google Maps static map using the specified parameters. No information is returned.',
    'parameters': {
      'type': 'OBJECT',
      'properties': {
        'center': {
            'type': 'STRING',
            'description': 'Location to center the map. It can be a lat,lng pair (e.g. 40.714728,-73.998672), or a string address of a location (e.g. Berkeley,CA).',
        },
        'zoom': {
            'type': 'NUMBER',
            'description': 'Google Maps zoom level. 1 is the world, 20 is zoomed in to building level. Integer only. Level 11 shows about a 15km radius. Level 9 is about 30km radius.'
        },
        'path': {
            "type": "STRING",
            'description': """The path parameter defines a set of one or more locations connected by a path to overlay on the map image. The path parameter takes set of value assignments (path descriptors) of the following format:

path=pathStyles|pathLocation1|pathLocation2|... etc.

Note that both path points are separated from each other using the pipe character (|). Because both style information and point information is delimited via the pipe character, style information must appear first in any path descriptor. Once the Maps Static API server encounters a location in the path descriptor, all other path parameters are assumed to be locations as well.

Path styles
The set of path style descriptors is a series of value assignments separated by the pipe (|) character. This style descriptor defines the visual attributes to use when displaying the path. These style descriptors contain the following key/value assignments:

weight: (optional) specifies the thickness of the path in pixels. If no weight parameter is set, the path will appear in its default thickness (5 pixels).
color: (optional) specifies a color either as a 24-bit (example: color=0xFFFFCC) or 32-bit hexadecimal value (example: color=0xFFFFCCFF), or from the set {black, brown, green, purple, yellow, blue, gray, orange, red, white}.

When a 32-bit hex value is specified, the last two characters specify the 8-bit alpha transparency value. This value varies between 00 (completely transparent) and FF (completely opaque). Note that transparencies are supported in paths, though they are not supported for markers.

fillcolor: (optional) indicates both that the path marks off a polygonal area and specifies the fill color to use as an overlay within that area. The set of locations following need not be a "closed" loop; the Maps Static API server will automatically join the first and last points. Note, however, that any stroke on the exterior of the filled area will not be closed unless you specifically provide the same beginning and end location.
geodesic: (optional) indicates that the requested path should be interpreted as a geodesic line that follows the curvature of the earth. When false, the path is rendered as a straight line in screen space. Defaults to false.
Some example path definitions:

Thin blue line, 50% opacity: path=color:0x0000ff80|weight:1
Solid red line: path=color:0xff0000ff|weight:5
Solid thick white line: path=color:0xffffffff|weight:10
These path styles are optional. If default attributes are desired, you may skip defining the path attributes; in that case, the path descriptor's first "argument" will consist instead of the first declared point (location).

Path points
In order to draw a path, the path parameter must also be passed two or more points. The Maps Static API will then connect the path along those points, in the specified order. Each pathPoint is denoted in the pathDescriptor separated by the | (pipe) character.
""",
        },
        'markers': {
            "type": "ARRAY",
            "items": {
                "type": "STRING"
            },
            # Copied from https://developers.google.com/maps/documentation/maps-static/start#Markers
            'description': """The markers parameter defines a set of one or more markers (map pins) at a set of locations. Each marker defined within a single markers declaration must exhibit the same visual style; if you wish to display markers with different styles, you will need to supply multiple markers parameters with separate style information.

The markers parameter takes set of value assignments (marker descriptors) of the following format:

markers=markerStyles|markerLocation1| markerLocation2|... etc.

The set of markerStyles is declared at the beginning of the markers declaration and consists of zero or more style descriptors separated by the pipe character (|), followed by a set of one or more locations also separated by the pipe character (|).

Because both style information and location information is delimited via the pipe character, style information must appear first in any marker descriptor. Once the Maps Static API server encounters a location in the marker descriptor, all other marker parameters are assumed to be locations as well.

Marker styles
The set of marker style descriptors is a series of value assignments separated by the pipe (|) character. This style descriptor defines the visual attributes to use when displaying the markers within this marker descriptor. These style descriptors contain the following key/value assignments:

size: (optional) specifies the size of marker from the set {tiny, mid, small}. If no size parameter is set, the marker will appear in its default (normal) size.
color: (optional) specifies a 24-bit color (example: color=0xFFFFCC) or a predefined color from the set {black, brown, green, purple, yellow, blue, gray, orange, red, white}.

Note that transparencies (specified using 32-bit hex color values) are not supported in markers, though they are supported for paths.

label: (optional) specifies a single uppercase alphanumeric character from the set {A-Z, 0-9}. (The requirement for uppercase characters is new to this version of the API.) Note that default and mid sized markers are the only markers capable of displaying an alphanumeric-character parameter. tiny and small markers are not capable of displaying an alphanumeric-character.
""",
        }
      },
      "required": [
        "center",
        "zoom",
      ]

    },
  },
]

Now define the `draw_map` function and add `google_search` as a tool to use for this conversation. This will allow the model to look up restaurants that might be popular.

In [ ]:
from urllib.parse import urlencode

import altair as alt
from google.api_core import retry
import requests


def draw_map(center, zoom, path: str = "", markers: list[str] = ()):
  logger.debug(f'MAPS: {center=} {zoom=} {path=} {markers=}')
  q = {
      'key': MAPS_API_KEY,
      'size': '512x512',
      'center': center,
      'zoom': zoom,
  }

  if path:
    q['path'] = path

  qs = list(q.items())

  for marker in markers:
    qs.append(('markers', marker))

  url = f'https://maps.googleapis.com/maps/api/staticmap?{urlencode(qs)}'
  display.display(display.Image(url=url))
  logger.debug(f"Map URL: {url}")

  return {'string_value': f'ok'}


tool_calls = {
    'draw_map': draw_map,
}

tools = [
    {'google_search': {}},
    {'function_declarations': map_fns},
]

Finally, define and run the conversation.

In [ ]:
async def go():
  async with quick_connect(tools=tools, modality="TEXT") as ws:

    # Google Search + Tools (Maps)
    await run(ws, "Please look up and mark 3 Sydney restaurants that are currently trending on a map.", responses=tool_calls)
    # Code execution + Tools
    await run(ws, "Now write some code to randomly pick one to eat at tonight and zoom in to that one on the map.", responses=tool_calls)


logger.setLevel('INFO')
await go()

The output for the first image will look something like this. Don't worry if yours is slightly different, there are many popular restaurants, and many ways to style a map. You can always ask the model for more specific guidance if you wish.

![Map with 3 colored markers](https://storage.googleapis.com/generativeai-downloads/images/map-example-1.png)

### Maps with Code execution

In this example, you will use the Google Maps tools defined before, and you'll challenge the model to generate a color gradient and uses it to visually represent data on a map. This task requires code execution, so it is also included as a tool.

Specifically, you will ask the model to plot the capital cities in Australia, and apply a gradient between two colors in a circular direction around the country using Google Maps markers.

In [ ]:
from urllib.parse import urlencode

import altair as alt
from google.api_core import retry


tool_calls = {
    'draw_map': draw_map,
}

tools = [
    {'code_execution': {}},
    {'function_declarations': map_fns},
]

async def go():
  async with quick_connect(tools=tools, modality="TEXT") as ws:

    # Code exec and tool use. No search.
    await run(ws, "Plot markers on every capital city in Australia using a gradient between "
                  "Orange and Green. Plan out your steps first, then follow the plan.", responses=tool_calls)

    await run(ws, "Awesome! Can you ensure the gradient is applied smoothly in a circular direction "
                  "around the country?", responses=tool_calls)


logger.setLevel('INFO')
await go()

The final image should look something like this.

![Map of Australia with colored markers styled in a circular gradient](https://storage.googleapis.com/generativeai-downloads/images/map-example-2.png)

Performance in this example depends on your feedback to get the output perfect. This example showed the first 2 steps of a hypothetical conversation, but you could keep iterating with the model until the results are what you need.

## Next steps

<a name="next_steps"></a>

This guide shows more intermedite use of the Multimodal Live API over Websockets.

- To try multimodal streaming, use [Live API in Google AI Studio](https://aistudio.google.com/app/live) - no code required.
- Check out the [Live API starter using the Python SDK](../quickstarts/Get_started_LiveAPI.ipynb) or using [websockets](../quickstarts/websockets/Get_started_LiveAPI.ipynb).
- Try some other [grounding](../examples/Search_grounding_for_research_report.ipynb) examples.
- Try the [Tool use in the live API tutorial](../quickstarts/Get_started_LiveAPI_tools.ipynb) for a walkthrough of Gemini live tool use capabilities.

Or just check the other Gemini capabilities illustrated in the [Cookbook examples](https://github.com/google-gemini/cookbook/tree/main/examples/) .